In [1]:
# Clone Real-ESRGAN and enter the Real-ESRGAN
!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN
# Set up the environment
!pip install basicsr
!pip install facexlib
!pip install gfpgan
!pip install -r requirements.txt
!python setup.py develop

Cloning into 'Real-ESRGAN'...
remote: Enumerating objects: 755, done.
remote: Total 755 (delta 0), reused 0 (delta 0), pack-reused 755
Receiving objects: 100% (755/755), 5.37 MiB | 3.77 MiB/s, done.
Resolving deltas: 100% (410/410), done.
/content/Real-ESRGAN
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 83.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.7/254.7 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 75.4 MB/s eta 0:00:00
  Created wheel for basicsr: filename=basicsr-1.4.2-py3-none-any.whl size=214818 sha256=364eca81137162e7079fd06fac1a96a5204c79ebe93255691584302208c1d28c
  Stored in directory: /root/.cache/pip/wheels/38/83/99/2d8437cc652a01af27df5ff037a4075e95b52d67705c5f30ca
Successfully built basicsr
     ━

In [2]:
import os
from google.colab import files
import shutil

upload_folder = 'upload'
result_folder = 'results'

if not os.path.isdir(upload_folder):
    os.mkdir(upload_folder)

if not os.path.isdir(result_folder):
    os.mkdir(result_folder)

# upload images
uploaded = files.upload()
for filename in uploaded.keys():
  dst_path = os.path.join(upload_folder, filename)
  print(f'move {filename} to {dst_path}')
  shutil.move(filename, dst_path)

Saving Trial_2_medio.mp4 to Trial_2_medio.mp4
move Trial_2_medio.mp4 to upload/Trial_2_medio.mp4


In [3]:
import subprocess
import cv2
import matplotlib.pyplot as plt

# Función para extraer frames de un video
def extraer_frames(video_path, frames_dir):
    os.makedirs(frames_dir, exist_ok=True)
    vidcap = cv2.VideoCapture(video_path)
    success, image = vidcap.read()
    count = 0
    while success:
        cv2.imwrite(f"{frames_dir}/frame{count:04d}.png", image)
        success, image = vidcap.read()
        count += 1

In [4]:
def aplicar_real_esrgan(frames_originales):
    uploads_dir= 'upload'
    results_dir = 'results'
    frames_procesados = 'frames_procesados'
    os.makedirs(uploads_dir, exist_ok=True)
    os.makedirs(results_dir, exist_ok=True)
    os.makedirs(frames_procesados, exist_ok=True)

    # Crear una lista de archivos PNG en la carpeta 'frames_originales', ordenados alfabéticamente
    lista_png = sorted([f for f in os.listdir(frames_originales) if f.endswith('.png')])

    for frame in lista_png:
      if frame.endswith('.png'):
        original_frame_path = os.path.join(frames_originales, frame)
        upload_frame_path = os.path.join(uploads_dir, frame)

        # Mover la imagen a la carpeta 'uploads'
        shutil.move(original_frame_path, upload_frame_path)
        !python inference_realesrgan.py -n RealESRGAN_x4plus -i upload --outscale 3.5 --face_enhance
        # Mover la imagen la imagen
        # Crear la carpeta si no exisiste
        if not os.path.isdir(frames_procesados):
                  os.mkdir(frames_procesados)

        # mover todos los archivos png de upload a procesados
        for file in os.listdir(uploads_dir):
            if file.endswith(".png"):
                shutil.move(os.path.join(uploads_dir, file), frames_procesados)


        # Ejecutar Real-ESRGAN
        #os.system(f"python inference_realesrgan.py -n RealESRGAN_x4plus -i {uploads_dir} --outscale 3.5 --face_enhance")

In [5]:
# Función para reconstruir el video
def reconstruir_video(processed_frames_dir, output_video_path, fps=30):
    cmd = f"ffmpeg -framerate {fps} -i {processed_frames_dir}/frame%04d_out.png -c:v libx264 -pix_fmt yuv420p {output_video_path}"
    subprocess.call(cmd, shell=True)

In [ ]:
import cv2
import os
import shutil
import subprocess

# Directorios
upload_folder = 'upload'
result_folder = 'results'
frames_originales_dir = 'frames_originales'
frames_procesados_dir =result_folder
video_procesado = 'video_mejorado.mp4'

# Asegúrate de que los directorios existan
if not os.path.isdir(frames_originales_dir):
    os.mkdir(frames_originales_dir)
if not os.path.isdir(frames_procesados_dir):
    os.mkdir(frames_procesados_dir)


# Ruta del video
video_original = 'upload/Trial_2_medio.mp4'

# Verificar si el archivo existe
if os.path.exists(video_original):
    print("El archivo existe.")
else:
    print("El archivo no existe. Verifica la ruta.")


# Proceso
#extraer_frames(video_original, frames_originales_dir)
aplicar_real_esrgan(frames_originales_dir)
#reconstruir_video(result_folder, video_procesado)

El archivo existe.
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(
Downloading: "https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth" to /content/Real-ESRGAN/weights/RealESRGAN_x4plus.pth

100% 63.9M/63.9M [00:00<00:00, 369MB/s]
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the fu